Load the pdf

In [82]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader(r"D:\The Silent Patient.pdf")
pdf = loader.load()

In [83]:
len(pdf)

787

In [84]:
texts = " ".join(i.page_content for i in pdf)

In [85]:
len(texts)

1404407

Divide into chunks

In [86]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 200)
cnks = text_splitter.create_documents([texts])

In [87]:
len(cnks)

783

In [88]:
len(cnks[5].page_content)

1940

In [89]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

emb_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2" 
)

vector_space = Chroma.from_documents(
    documents= cnks,
    embedding=emb_model,
    persist_directory="vect_space"
)

vector_space.persist()

In [90]:
len(vector_space.get()["documents"][5])

9928

In [91]:
retriever = vector_space.as_retriever(
        search_type="similarity",
        search_kwargs={'k': 4}
    )

In [92]:
retriever.invoke("who is the author of the book a silent patient")

[Document(metadata={}, page_content='She told the story of a doctor who had the Pﬁzer jab and when\nquestioned had no idea what was in it. The doctor had never read\nthe literature. We have to stop treating doctors as intellectual giants\nwhen so many are moral and medical pygmies. The doctor did not\neven know that the ‘vaccines’ were not fully approved or that their\ntrials were ongoing. They were, however, asking their patients if\nthey minded taking part in follow-ups for research purposes – yes,\nthe ongoing clinical trial. The nurse said the doctor’s ignorance was\nnot rare and she had spoken to a hospital consultant who had the jab\nwithout any idea of the background or that the ‘trials’ had not been\ncompleted. Nurses and pharmacists had shown the same ignorance. ‘My NHS colleagues have forsaken their duty of care, broken their\ncode of conduct – Hippocratic Oath – and have been brainwashed\njust the same as the majority of the UK public through propaganda\n…’ She said she had 

In [93]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model

In [94]:
os.environ.get("GROQ_API_KEY")
model = init_chat_model("groq:llama-3.1-8b-instant")

In [95]:
from langchain.prompts import PromptTemplate

In [96]:
prompt = PromptTemplate(
    template = """
            You are a helpfull ai assistent. answer according to the provided context. if the context is insufficient, just say you dont know.

            context = {context},
            \n\n
            question = {question}

""",
    input_variables=["context","question"]  
)

In [97]:
question = "tell me about the characteristics of the patient mentioned in this book"
retrieved_dict = retriever.invoke(question)
retrieved_context = " ".join(i.page_content for i in retrieved_dict)

In [98]:
len(retrieved_context)

7857

In [99]:
main_prompt = prompt.invoke({"context": retrieved_context, "question": question})

In [100]:
resp = model.invoke(main_prompt)

In [101]:
print(resp)

content="Based on the provided context, the patient mentioned is Alicia Berenson. Here are some characteristics deduced from the information:\n\n1. **Self-harming behavior**: Alicia has a history of slashing her wrists twice and engaging in self-harm with various objects upon admission.\n2. **Aggressive behavior**: Alicia became violent in the canteen, smashing a plate and attempting to slash Elif's throat with a jagged edge.\n3. **Withdrawn and isolated**: Alicia made no effort to interact with patients or staff, remaining withdrawn and isolated. She quickly became invisible to the other patients.\n4. **Violent outburst**: Alicia's confrontation with Elif in the canteen escalated rapidly, requiring restraint, sedation, and placement in isolation.\n5. **Lack of communication**: Alicia didn't respond to attempts at conversation and never initiated any interactions with her peers or staff.\n\nThese characteristics suggest that Alicia may be struggling with severe mental health issues, su